In [1]:
import random
import numpy as np
import pandas as pd 
import networkx as nx 
from tqdm import tqdm
from gensim.models import Word2Vec

from alias import create_alias_tables, alias_sample
from feature_column import SparseFeat

import tensorflow as tf
from tensorflow.python.keras.initializers import RandomNormal, Zeros, glorot_normal
from tensorflow.keras.layers import Layer
from tensorflow.python.keras.regularizers import l2
from tensorflow.keras.layers import Flatten, Concatenate, Dense, Reshape

from features import FeatureEncoder
from utils import *
from tensorflow.keras.models import Model
from tensorflow.python.keras import backend as K
from sklearn.preprocessing import LabelEncoder

In [2]:
# 带权的deepwalk随机游走
def deepwalk_weight(G, walk_length=10):
    """
    1. 遍历图中所有节点，以每个节点为起始节点采样
    2. 用带权邻居采样代替随机采样
    """
    nodes_list = G.nodes
    
    # 处理每个节点邻居节点的采样概率，并转化成alias表的形式
    node_alias_dict = {}
    for node in tqdm(nodes_list):
        nbrs_prob_list = [G[node][nbr].get('weight', 1.0) \
                          for nbr in list(G.neighbors(node))]
        normalized_prob_list = [float(prob) / sum(nbrs_prob_list)\
                               for prob in nbrs_prob_list]
        # 构建alias表
        node_alias_dict[node] = create_alias_tables(normalized_prob_list)
    
    # 遍历每个节点，以每个节点为起始节点进行采样
    sentenses_list = []
    for node in tqdm(nodes_list):
        tmp_sentence_list = [node]
        
        while len(tmp_sentence_list) < walk_length:
            cur_node = tmp_sentence_list[-1]
            nbrs_list = list(G.neighbors(cur_node))
            
            # 如果当前节点没有邻居节点，则停止采样
            if len(nbrs_list) == 0:
                break
            
            # 获取当前节点邻居节点的alias表
            accept_prob = node_alias_dict[cur_node][0]
            alias_table = node_alias_dict[cur_node][1]
            sample_index = alias_sample(accept_prob, alias_table)
            tmp_sentence_list.append(nbrs_list[sample_index])
        
        sentenses_list.append(tmp_sentence_list)
    
    return sentenses_list

In [3]:
def get_all_pairs(sentences, window_size=5):
    """
    1. 遍历所有的序列，根据设定的窗口，获取共现的pair
    """
    
    all_pairs_list = []
    
    # 遍历所有序列
    for sentence in tqdm(sentences):
        # 遍历序列的每个位置
        for i in range(len(sentence)):
            # 在当前元素的前、后窗口中构造pair
            for j in range(i - window_size, i + window_size + 1):
                # 如果窗口中的元素等于当前元素，或者当前窗口不合法
                if i == j or j < 0 or j >= len(sentence):
                    continue
                # 构造pair
                all_pairs_list.append((sentence[i], sentence[j]))
                
    # 用np规整一下所有的格式，防止索引中存在float类型的数据
    return np.array(all_pairs_list, dtype=np.int32)

In [4]:
def get_train_data(all_paris):
    # 构造训练样本，就是把side info考虑进去
    # 数据格式：  side_info_sku_id, brand, cate, shop_id, sku_id  (no side_info)
    sku_side_info_df = pd.read_csv('./sku_sideinfo.csv', dtype=np.int)

    all_pairs_df = pd.DataFrame(all_paris, columns=['side_info_sku_id', 'sku_id_pair'], dtype=np.int)

    train_df = pd.merge(all_pairs_df, sku_side_info_df, 
                        left_on='side_info_sku_id', right_on='sku_id')

    # 对sku的side info类别编码
    for side_info in ['brand', 'shop_id', 'cate']:
        lbe = LabelEncoder()
        train_df[side_info] = lbe.fit_transform(train_df[side_info])
    
    feature_max_index_dict = {}
    feature_max_index_dict['side_info_sku_id'] = train_df['side_info_sku_id'].max() + 1
    feature_max_index_dict['brand'] = train_df['brand'].max() + 1
    feature_max_index_dict['shop_id'] = train_df['shop_id'].max() + 1
    feature_max_index_dict['cate'] = train_df['cate'].max() + 1
    feature_max_index_dict['sku_id_pair'] = train_df['sku_id_pair'].max() + 1

    feature_names = ['side_info_sku_id', 'brand', 'shop_id', 'cate', 'sku_id_pair']
    train_input_dict = {}
    for name in feature_names:
        train_input_dict[name] = np.array(train_df[name].values) 

    train_label = np.array([1] * len(train_input_dict))

    return feature_max_index_dict, train_input_dict, train_label

In [5]:
# 读取图数据
G = nx.read_edgelist('./graph.csv', create_using=nx.DiGraph(), \
                     nodetype=None, data=[('weight', int)])
# 随机游走获取新的序列
sentences = deepwalk_weight(G)
# 过滤长度小于2的序列
sentences = [s for s in sentences if len(s) >= 2]

# 根据序列构造pairs
all_paris = get_all_pairs(sentences, window_size=5)

feature_max_index_dict, train_input_dict, train_label = get_train_data(all_paris)

100%|████████████████████████████████████████████████████████████████████████████████████| 27095/27095 [00:00<00:00, 50486.89it/s]
/tmp/ipykernel_21729/2579947807.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  sku_side_info_df = pd.read_csv('./sku_sideinfo.csv', dtype=np.int)
/tmp/ipykernel_21729/2579947807.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64

In [6]:
def EGES(sparse_feature_columns, sideinfo_features, 
         item_pair_feature_name,num_sampled=5):
    """EGES模型
    """
    # 获取所有的feature_columns
    feature_columns = sparse_feature_columns
    # 根据feature_colimns 构建input层和Embedding层，并将input层和Embedding层串起来
    feature_encode = FeatureEncoder(feature_columns)
    # 获取模型的输入，其实就是所有的Input层
    feature_input_layers_list = list(feature_encode.feature_input_layer_dict.values())
        
    # 将所有的sparse特征拿出来
    group_embedding_dict = feature_encode.sparse_feature_dict
        
    # 将所有的side info特征拿出来
    side_info_embedding_list = [v for k, v in group_embedding_dict['default_group'].items() 
        if k in sideinfo_features]
        
    # 把sku embedding层拿出来, 就是后面那个大矩阵层
    sku_embedding_layer = feature_encode.embedding_layers_dict['sku_id_pair']
    
    # 方便后面获取注意力权重
    side_info_sku_input_layer = feature_input_layers_list[0]
    
    # 注意只有一个
    item_pair_input = feature_encode.feature_input_layer_dict['sku_id_pair']
        
    item_vocabulary_size = sparse_feature_columns[0].vocabulary_size 
    embedding_dim = sparse_feature_columns[0].embedding_dim
    
    # (B, embedding_dim)
    side_info_pooling_output = EGESPooling(item_vocabulary_size, len(sideinfo_features))\
                    ([side_info_embedding_list, side_info_sku_input_layer])
        
    # 把所有的item 的embedding都拿出来
    item_index = EmbeddingIndex(list(range(item_vocabulary_size)))(feature_input_layers_list[0])
    item_embedding_weight = NoMask()(sku_embedding_layer(item_index))

    softmax_input = (item_embedding_weight, side_info_pooling_output, item_pair_input)
    
    # 采样的softmax
    output = SampledSoftmaxLayer(num_sampled)(softmax_input)
    
    model = Model(feature_input_layers_list, output)
    return model

In [ ]:
# 构造模型

embedding_dim = 16
side_info_feature_name = ['side_info_sku_id', 'brand', 'shop_id', 'cate']
item_pair_feature_name = ['sku_id_pair']

sparse_feature_columns = [SparseFeat(name, vocabulary_size=feature_max_index_dict[name], \
                        embedding_dim=embedding_dim, embedding_name=name) for name in \
                        side_info_feature_name + item_pair_feature_name]

model = EGES(sparse_feature_columns, side_info_feature_name, item_pair_feature_name)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), \
              loss=sampledsoftmaxloss)

In [8]:
model.fit(train_input_dict, train_label, batch_size=1024, epochs=2, verbose=1, validation_split=0.2)

Epoch 1/2


/home/ryluo/anaconda3/envs/tf2.2/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
2022-05-02 18:56:17.461291: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


1007/1007 [==============================] - 12s 12ms/step - loss: 14.7934 - val_loss: 2.1919
Epoch 2/2
1007/1007 [==============================] - 11s 11ms/step - loss: 1.3425 - val_loss: 1.0312
